In [38]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [39]:
gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))
print("Num GPUs Available: ", gpus_available)
assert gpus_available > 0

Num GPUs Available:  1


In [40]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
import pandas as pd
import numpy as np

In [42]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [43]:
import tensorflow_datasets as tfds
df_amazon_review = tfds.load('amazon_us_reviews/Mobile_Electronics_v1_00', split = 'train',shuffle_files = True)
assert isinstance(df_amazon_review, tf.data.Dataset)
# print(df_amazon_review)

In [44]:
df_new = tfds.as_dataframe(df_amazon_review)

In [45]:
df_new.head(5)

,data/customer_id,data/helpful_votes,data/marketplace,data/product_category,data/product_id,data/product_parent,data/product_title,data/review_body,data/review_date,data/review_headline,data/review_id,data/star_rating,data/total_votes,data/verified_purchase,data/vine
0,b'20980074',0,b'US',b'Mobile_Electronics',b'B00D1847NE',b'274617424',b'Teenage Mutant Ninja Turtles Boombox CD Play...,b'Does not work',b'2015-01-09',b'One Star',b'R1OVS0D6SEXPW7',1,0,0,1
1,b'779273',0,b'US',b'Mobile_Electronics',b'B00KMO6DYG',b'397452138',b'4 Gauge Amp Kit Amplifier Install Wiring Com...,b'This is a great wiring kit i used it to set ...,b'2015-08-06',b'Great kit',b'R9VSD0ET8FERB',4,0,0,1
2,b'15410531',0,b'US',b'Mobile_Electronics',b'B000GWLL0K',b'948304826',b'Travel Wall Charger fits Creative Zen Vision...,b'It works great so much faster than USB charg...,b'2007-03-15',b'A/C Charger for Creative Zen Vision M',b'R3ISXCZHWLJLBH',5,0,0,1
3,b'27389005',0,b'US',b'Mobile_Electronics',b'B008L3JE6Y',b'466340015',b'High Grade Robust 360\xc2\xb0 Adjustable Car...,b'This product was purchased to hold a monitor...,b'2013-07-30',b'camera stand',b'R1TWVUDOFJSQAW',5,0,0,1
4,b'2663569',0,b'US',b'Mobile_Electronics',b'B00GHZS4SC',b'350592810',b'HDE Multifunctional Bluetooth FM Audio Car K...,"b""it works but it has really bad sound quality...",b'2014-12-31',b'bad sound quality',b'R2PEOEUR1LP0GH',3,0,0,1


In [46]:
df_new["Sentiment"] = df_new["data/star_rating"].apply(lambda score: "positive" if score >= 3 else "negative")
df_new["Sentiment"] = df_new['Sentiment'].map({'positive':1, 'negative':0})
df_new['short_review'] =df_new['data/review_body'].str.decode("utf-8")
df_new = df_new[["short_review", "Sentiment"]]

In [47]:
df_new.head()

,short_review,Sentiment
0,Does not work,0
1,This is a great wiring kit i used it to set up...,1
2,It works great so much faster than USB charger...,1
3,This product was purchased to hold a monitor o...,1
4,it works but it has really bad sound quality. ...,1


In [48]:
n = 54975
df_new.drop(df_new.tail(n).index,inplace = True)

<ipython-input-48-d39cdf0e11dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.drop(df_new.tail(n).index,inplace = True)


In [49]:
r = df_new['short_review'].values.tolist()
labels = df_new['Sentiment'].tolist()

In [50]:
train_sentences, validate_sentences, training_labels, validation_labels = train_test_split(r, labels, test_size=0.2)

In [51]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [52]:
tokenizer([train_sentences[0]], truncation=True,padding=True, max_length=128)

{'input_ids': [[101, 1045, 2359, 2242, 12283, 1012, 2025, 4206, 2438, 2000, 4906, 2026, 6207, 1020, 4606, 3042, 1999, 2026, 7961, 15007, 25776, 2276, 999, 3098, 2003, 2000, 4867, 1012, 1026, 7987, 1013, 1028, 2515, 2025, 2147, 999, 2005, 2033, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [53]:
train_encodings = tokenizer(train_sentences,truncation=True,padding=True)
val_encodings = tokenizer(validate_sentences,truncation=True,padding=True)

In [54]:
training_datasets = tf.data.Dataset.from_tensor_slices((dict(train_encodings),training_labels))

In [55]:
validate_datasets = tf.data.Dataset.from_tensor_slices((dict(val_encodings),validation_labels))

In [56]:
m = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use i

In [57]:
optimize = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
m.compile(optimizer=optimize, loss=m.compute_loss, metrics=['accuracy'])

In [58]:
m.save_pretrained("./sentimental_analysis")

In [66]:
loaded_model = TFDistilBertForSequenceClassification.from_pretrained("/content/sentimental_analysis/") #/content/sentiment/config.json

Some layers from the model checkpoint at /content/sentimental_analysis/ were not used when initializing TFDistilBertForSequenceClassification: ['dropout_79']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/sentimental_analysis/ and are newly initialized: ['dropout_119']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [67]:
test_sentence = 'This is a really good product. I love it.'

In [68]:
predict_input = tokenizer.encode(test_sentence,truncation=True, padding=True,return_tensors="tf")

In [71]:
tf_output = loaded_model.predict(predict_input)[0]
tf_predict = tf.nn.softmax(tf_output, axis=1)
labels = ['Negative','Positive']
label = tf.argmax(tf_predict, axis=1)
label = label.numpy()
print(labels[label[0]])

1/1 [==============================] - 0s 37ms/step
Positive
